In [ ]:
import scipy.stats as st
st.pearsonr(fragment_x, fragment_y)

In [ ]:
50Hz
segement count = Frame count + 1
-> varying segment length

frame length = 2 * segment length
segment length = sample size / segment count
cut off = sample size % segment count

segment length should be at least 5
2,5sec -> 125 samples
5 = 125 / segment count <=> framecount + 1 = 125 / 5 <=> framecount = 25 - 1 = 24
10 = 125 / segment count <=> framecount + 1 = 125 / 10 <=> framecount = 12.5 - 1 = 11.5

In [2]:
18

3 3 3 3 3 3

33 33 33 33 33

11
12


In [8]:
x = list(range(1,21))
print(x)
segment_count = 5 + 1
segment_length = len(x) // segment_count
print(segment_length)
cut_off = len(x) % segment_count
print(x[:-cut_off])
x = x[:-cut_off]
frame_length = 2 * segment_length

for segment in range(0, len(x) - segment_length, segment_length):
    step_size = segment + frame_length
    print(x[segment:step_size])

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
3
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
[1, 2, 3, 4, 5, 6]
[4, 5, 6, 7, 8, 9]
[7, 8, 9, 10, 11, 12]
[10, 11, 12, 13, 14, 15]
[13, 14, 15, 16, 17, 18]


In [ ]:
x, y, z => xy xz yz
x, y, z, w => xy xz xw yz yw zw

In [1]:
import numpy as np
import scipy.spatial.distance as dis
import scipy.stats as st

def calculate_similarity(a, b, sim_type):
    if sim_type == 'cosine':
        return np.dot(a,b)/(np.linalg.norm(a)*np.linalg.norm(b))
    elif sim_type == 'euclid':
        return 1/(1+dis.euclidean(a, b))
    elif sim_type == 'pearson':
        return st.pearsonr(a, b)[0]
    else:
        print('sim_type value "{}" is not supported, choose cosine, eucild or pearson instead'.format(sim_type))

def calculate_distance(a, b, dist_type):
    if dist_type == 'cosine':
        return 1 - np.dot(a,b)/(np.linalg.norm(a)*np.linalg.norm(b))
    elif dist_type == 'euclid':
        return dis.euclidean(a, b)
    elif dist_type == 'pearson':
        return 1 - st.pearsonr(a, b)[0]
    else:
        print('sim_type value "{}" is not supported, choose cosine, eucild or pearson instead'.format(sim_type))

In [14]:
x = [1,0]
y = [1,1]

print('cosine: {}'.format(calculate_similarity(x,y, 'cosine')))
print('euclid: {}'.format(calculate_similarity(x,y, 'euclid')))
print('pearson: {}'.format(calculate_similarity(x,y, 'pearson')))

cosine: 0.7071067811865475
euclid: 0.5
pearson: nan


c:\python36\lib\site-packages\scipy\stats\stats.py:3021: RuntimeWarning: invalid value encountered in double_scalars
  r = r_num / r_den
c:\python36\lib\site-packages\scipy\stats\stats.py:5277: RuntimeWarning: invalid value encountered in less
  x = np.where(x < 1.0, x, 1.0)  # if x > 1 then return 1.0
